In [1]:
from unsloth import FastLanguageModel 
import torch
from peft import PeftModel, PeftConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/mnt/bn/data-tns-live-llm/leon/datasets/fed/niid_med_0.01_20000_fedavg_c10s2_i20_b16a1_l2048_r128a256_f0",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)
# model = PeftModel.from_pretrained(model,"/mnt/bn/data-tns-live-llm/leon/datasets/pos")
print(model)

==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.109 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 9.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [3]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [4]:
inputs = tokenizer("generate some instruction tuning sample used for your training", return_tensors="pt")
input_ids = inputs.input_ids.cuda()
generate_ids = model.generate(input_ids, max_length=2048, repetition_penalty=1.1, streamer = text_streamer)
outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(outputs)

<s> generate some instruction tuning sample used for your training. савезни јавни радио и телевизија 2014.
generate some instruction tuning sample used for your training.</s>
generate some instruction tuning sample used for your training. савезни јавни радио и телевизија 2014.
generate some instruction tuning sample used for your training.


# 上传模型

In [18]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = "LEON24/files"
folder_path = "/mnt/bn/data-tns-live-llm/leon/datasets/files"
api.create_repo(repo_id, exist_ok=True, private=True)
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_id,
)

code_cherry_1000.pt:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

code_MoDS_1000.pt:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

MathInstruct.json:   0%|          | 0.00/212M [00:00<?, ?B/s]

cherry_1000.pt:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

MoDS_1000.pt:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

code_data_cherry.pt:   0%|          | 0.00/25.8M [00:00<?, ?B/s]

math_data_cherry.pt:   0%|          | 0.00/38.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LEON24/files/commit/6463c78cb88699b514be76499756d33d9c6bcc64', commit_message='Upload folder using huggingface_hub', commit_description='', oid='6463c78cb88699b514be76499756d33d9c6bcc64', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, T5ForConditionalGeneration
import torch
model = T5ForConditionalGeneration.from_pretrained("google/t5-xl-lm-adapt", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/t5-xl-lm-adapt")

/home/tiger/.local/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# 调试用参数

In [8]:
args = """--model_name_or_path google/t5-xl-lm-adapt --data_path /mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet --per_device_train_batch_size 256 --eval_strategy no --num_train_epochs 3 --learning_rate 2e-5 --warmup_ratio 0.03 --output_dir /mnt/bn/data-tns-live-llm/leon datasets/p3_exp1 --logging_steps 100 --save_strategy "steps" --save_steps 100 --save_total_limit 3 --model_max_length 1024""".split(" ")
print(args)

['--model_name_or_path', 'google/t5-xl-lm-adapt', '--data_path', '/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet', '--per_device_train_batch_size', '256', '--eval_strategy', 'no', '--num_train_epochs', '3', '--learning_rate', '2e-5', '--warmup_ratio', '0.03', '--output_dir', '/mnt/bn/data-tns-live-llm/leon', 'datasets/p3_exp1', '--logging_steps', '100', '--save_strategy', '"steps"', '--save_steps', '100', '--save_total_limit', '3', '--model_max_length', '1024']


In [10]:
print("['--model_name_or_path', 'google/t5-xl-lm-adapt', '--data_path', '/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet', '--per_device_train_batch_size', '256', '--eval_strategy', 'no', '--num_train_epochs', '3', '--learning_rate', '2e-5', '--warmup_ratio', '0.03', '--output_dir', '/mnt/bn/data-tns-live-llm/leon', 'datasets/p3_exp1', '--logging_steps', '100', '--save_strategy', 'steps', '--save_steps', '100', '--save_total_limit', '3', '--model_max_length', '1024']".replace("\'","\""))

["--model_name_or_path", "google/t5-xl-lm-adapt", "--data_path", "/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet", "--per_device_train_batch_size", "256", "--eval_strategy", "no", "--num_train_epochs", "3", "--learning_rate", "2e-5", "--warmup_ratio", "0.03", "--output_dir", "/mnt/bn/data-tns-live-llm/leon", "datasets/p3_exp1", "--logging_steps", "100", "--save_strategy", "steps", "--save_steps", "100", "--save_total_limit", "3", "--model_max_length", "1024"]


In [2]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

# 合并模型

In [47]:
from transformers import AutoConfig, T5ForConditionalGeneration, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import torch
import inspect
from accelerate import Accelerator
from peft import PeftModel
accelerator = Accelerator()
base_path = "google/t5-xl-lm-adapt"
source_path = "/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1"
target_path = "/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp2_merged/"
# config = AutoConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1")
model = AutoModelForSeq2SeqLM.from_pretrained(base_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(base_path)
model:PeftModel = PeftModel.from_pretrained(model, source_path)
print(model)
# model = model.merge_and_unload()
# print(model)
# model.save_pretrained(target_path)
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_path)
# tokenizer.save_pretrained(target_path)
# signature = inspect.signature(model.forward)
# signature_columns = list(signature.parameters.keys())
# print(signature_columns)

Detected kernel version 5.4.143, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


SafetensorError: Error while deserializing header: InvalidHeaderDeserialization

In [ ]:
config = AutoConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1_merged")
print(config)

In [46]:
from transformers import BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
model: T5ForConditionalGeneration = T5ForConditionalGeneration.from_pretrained("google/t5-xl-lm-adapt", quantization_config=quantization_config)

model = prepare_model_for_kbit_training(model)
tokenizer = AutoTokenizer.from_pretrained("google/t5-xl-lm-adapt", model_max_length=1024)

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=32,
        bias="none",
        task_type="SEQ_2_SEQ_LM",
        # target_modules= ['v', 'o'],
)

model = get_peft_model(model, peft_config)
print(model)
model.print_trainable_parameters()

`low_cpu_mem_usage` was None, now set to True since model is quantized.


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 2048)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 2048)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict(

In [45]:
target_path = "/mnt/bn/data-tns-live-llm/leon/datasets/p3_tmp"
model.merge_adapter()
print(model)
model.save_pretrained(target_path)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(base_path)
tokenizer.save_pretrained(target_path)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 2048)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 2048)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear4bit(in_features=2048, out_features=2048, bias=False)
                  (k): Linear4bit(in_features=2048, out_features=2048, bias=False)
                  (v): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDic

('/mnt/bn/data-tns-live-llm/leon/datasets/p3_tmp/tokenizer_config.json',
 '/mnt/bn/data-tns-live-llm/leon/datasets/p3_tmp/special_tokens_map.json',
 '/mnt/bn/data-tns-live-llm/leon/datasets/p3_tmp/spiece.model',
 '/mnt/bn/data-tns-live-llm/leon/datasets/p3_tmp/added_tokens.json',
 '/mnt/bn/data-tns-live-llm/leon/datasets/p3_tmp/tokenizer.json')

In [ ]:

peft_config = PeftConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/fed/niid_pos_public_20000_fedavg_c10s2_i10_b16a2_l2048_r16a16/")